# Equity Builder Demo

Демонстрационный ноутбук для загрузки логов тестирования, расчёта эквити и визуализации результатов.

In [24]:
# Импорты и настройка
import sys
sys.path.append('src')
from pathlib import Path

from data import CSVLoader
import pandas as pd
import numpy as np

from equity_builder import (
    load_log,
    select_last_test,
    log_to_trades,
    equity_on_events,
    equity_on_ohlc,
    EquityConfig,
)
from lightweight_charts import JupyterChart
from IPython.display import display

In [25]:
# Конфигурация данных
DATA_DIR = Path(r'G:\My Drive\data_fut')
LOG_DIR = Path(r'C:\Quik_Alor\lua\logs')

LOG_FILES = [
    'IMOEX_test1_8_1t.csv',
    # 'IMOEX_test1_8_2t.csv',
    # 'IMOEX_test1_8_3t.csv',
    # 'IMOEX_test1_8_4t.csv',
    # 'IMOEX_test1_8_5t.csv',
    # 'IMOEX_test1_8_6t.csv',
]

FILE_TICKER_MAP = {
    'IMOEX_test1_8_1t.csv': 'IMOEXF',
    # 'IMOEX_test1_8_2t.csv': 'IMOEXF',
    # 'IMOEX_test1_8_3t.csv': 'IMOEXF',
    # 'IMOEX_test1_8_4t.csv': 'IMOEXF',
    # 'IMOEX_test1_8_5t.csv': 'IMOEXF',
    # 'IMOEX_test1_8_6t.csv': 'IMOEXF',
}

TIMEFRAME = 5
combine_on_one_canvas = True
cfg = EquityConfig(point_value=1.0, commission_per_contract=0.0)

loader = CSVLoader(str(DATA_DIR), use_cache=True)

In [26]:
# Загрузка контрольных данных
TICKER = FILE_TICKER_MAP[LOG_FILES[0]]
start_date = '2025-09-30'
end_date = '2025-10-15'
timeframe = TIMEFRAME

control_df = loader.load(
    TICKER,
    timeframe=timeframe,
    start_date=start_date,
    end_date=end_date,
)
print(f'Данные: {len(control_df)} баров, период: {control_df.index[0]} - {control_df.index[-1]}')

Данные: 2383 баров, период: 2025-09-30 09:00:00 - 2025-10-14 23:50:00


In [27]:

# Вспомогательные функции
from collections import OrderedDict

COLOR_SET = [
    '#4e79a7', '#f28e2b', '#e15759', '#76b7b2',
    '#59a14f', '#edc948', '#b07aa1', '#ff9da7',
]

results = OrderedDict()

EVENT_COLUMNS = ['datetime', 'price', 'pos', 'avg_price', 'realized', 'mtm', 'equity']
MINUTE_COLUMNS = ['datetime', 'close', 'pos', 'avg_price', 'realized', 'mtm', 'equity']


def prepare_ohlc_frame(raw_df: pd.DataFrame) -> pd.DataFrame:
    """Format CSVLoader data with an explicit datetime column."""
    if raw_df is None or raw_df.empty:
        return pd.DataFrame(columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    ohlc = raw_df.copy()
    if not isinstance(ohlc.index, pd.DatetimeIndex):
        ohlc.index = pd.to_datetime(ohlc.index)
    ohlc = ohlc.sort_index()

    index_name = ohlc.index.name
    ohlc = ohlc.reset_index()

    if 'datetime' not in ohlc.columns:
        if index_name and index_name in ohlc.columns:
            ohlc = ohlc.rename(columns={index_name: 'datetime'})
        elif 'index' in ohlc.columns:
            ohlc = ohlc.rename(columns={'index': 'datetime'})
        else:
            first_col = ohlc.columns[0]
            ohlc = ohlc.rename(columns={first_col: 'datetime'})

    ohlc['datetime'] = pd.to_datetime(ohlc['datetime'], errors='coerce')
    ohlc = ohlc.dropna(subset=['datetime'])

    rename_map = {
        'OPEN': 'open',
        'Open': 'open',
        'HIGH': 'high',
        'High': 'high',
        'LOW': 'low',
        'Low': 'low',
        'CLOSE': 'close',
        'Close': 'close',
        'VOL': 'volume',
        'Volume': 'volume',
    }
    ohlc = ohlc.rename(columns={k: v for k, v in rename_map.items() if k in ohlc.columns})
    return ohlc


def process_log_file(
    file_name: str,
    method: str = 'topdown',
    ohlc_range: tuple[str | None, str | None] | None = None,
):
    """Load a log, isolate the last test and compute equity."""
    ticker = FILE_TICKER_MAP.get(file_name)
    if ticker is None:
        raise KeyError(f'Не указан тикер для файла {file_name}')

    log_path = LOG_DIR / file_name
    if not log_path.exists():
        print(f'{file_name}: файл не найден по пути {log_path}')
        return None

    log_df = load_log(log_path)
    last_df = select_last_test(log_df, method=method)

    valid_times = last_df['datetime'].dropna()
    if valid_times.empty:
        print(f'{file_name}: отсутствуют отметки времени в последнем тесте')
        return None

    start_dt = valid_times.min()
    end_dt = valid_times.max()
    trade_start_date = start_dt.date().isoformat()
    trade_end_date = end_dt.date().isoformat()

    load_start, load_end = (None, None)
    if ohlc_range is not None:
        load_start, load_end = ohlc_range
    load_start = load_start or trade_start_date
    load_end = load_end or trade_end_date

    trades = log_to_trades(last_df)
    events_equity = (
        equity_on_events(trades, cfg).sort_values('datetime').reset_index(drop=True)
        if not trades.empty
        else pd.DataFrame(columns=EVENT_COLUMNS)
    )

    try:
        ohlc_raw = loader.load(
            ticker,
            timeframe=TIMEFRAME,
            start_date=load_start,
            end_date=load_end,
        )
    except Exception as exc:
        print(f'{file_name}: не удалось загрузить данные {ticker} — {exc}')
        ohlc_raw = pd.DataFrame()

    ohlc_frame = prepare_ohlc_frame(ohlc_raw)

    minute_input = pd.DataFrame(columns=['datetime', 'close'])
    if 'close' in ohlc_frame.columns:
        minute_input = ohlc_frame[['datetime', 'close']].dropna().sort_values('datetime').copy()

    if minute_input.empty:
        minute_equity = pd.DataFrame(columns=MINUTE_COLUMNS)
    elif trades.empty:
        minute_equity = minute_input.assign(
            pos=0.0,
            avg_price=0.0,
            realized=0.0,
            mtm=0.0,
            equity=0.0,
        )[MINUTE_COLUMNS].reset_index(drop=True)
    else:
        minute_equity = equity_on_ohlc(trades, minute_input, cfg).reindex(columns=MINUTE_COLUMNS)
        minute_equity = minute_equity.sort_values('datetime').reset_index(drop=True)

    payload = {
        'file': file_name,
        'ticker': ticker,
        'log': last_df,
        'trades': trades,
        'events_equity': events_equity,
        'minute_equity': minute_equity,
        'ohlc': ohlc_frame,
        'start_dt': start_dt,
        'end_dt': end_dt,
        'start_date': trade_start_date,
        'end_date': trade_end_date,
        'load_start': load_start,
        'load_end': load_end,
    }
    return payload


In [28]:

# Обработка логов и расчёт эквити
results.clear()
global_ohlc_range = (start_date, end_date)
for idx, file_name in enumerate(LOG_FILES):
    payload = process_log_file(file_name, method='topdown', ohlc_range=global_ohlc_range)
    if payload is None:
        continue
    results[file_name] = payload

    trades_count = len(payload['trades'])
    events_points = len(payload['events_equity'])
    minute_points = len(payload['minute_equity'])
    trade_window = f"{payload['start_dt']:%Y-%m-%d %H:%M:%S} – {payload['end_dt']:%Y-%m-%d %H:%M:%S}"
    ohlc_window = f"{payload['load_start']} – {payload['load_end']}"
    print(
        f"{file_name} → {payload['ticker']} | трейды: {trade_window} | OHLC: {ohlc_window} | сделок: {trades_count} | event pts: {events_points} | minute pts: {minute_points}"
    )
    if minute_points == 0 and trades_count > 0:
        print(f"{file_name}: минутная эквити не построена (нет данных OHLC)")

print(f'Всего обработано: {len(results)} лог(ов)')


IMOEX_test1_8_1t.csv → IMOEXF | трейды: 2025-09-30 17:56:15 – 2025-10-29 02:00:05 | OHLC: 2025-09-30 – 2025-10-15 | сделок: 100 | event pts: 100 | minute pts: 2383
Всего обработано: 1 лог(ов)


In [29]:

# Визуализация эквити
last_payload = None


def build_line_frame(data: pd.DataFrame, value_col: str, label: str) -> pd.DataFrame | None:
    if data is None or data.empty or value_col not in data.columns:
        return None
    frame = (
        data[['datetime', value_col]]
        .rename(columns={'datetime': 'time', value_col: label})
        .dropna(subset=['time'])
        .sort_values('time')
        .drop_duplicates(subset=['time'], keep='last')
        .reset_index(drop=True)
    )
    frame['time'] = pd.to_datetime(frame['time'])
    return frame


def add_equity_series(chart_obj: JupyterChart, payload: dict, idx: int) -> int:
    ticker = payload['ticker']
    base_color = COLOR_SET[idx % len(COLOR_SET)]
    lines_added = 0

    events_label = f"{ticker} equity (events)"
    events_frame = build_line_frame(payload['events_equity'], 'equity', events_label)
    if events_frame is not None:
        events_line = chart_obj.create_line(name=events_label, color=base_color, width=2)
        events_line.set(events_frame)
        lines_added += 1

    minute_label = f"{ticker} equity (ohlc)"
    minute_frame = build_line_frame(payload['minute_equity'], 'equity', minute_label)
    if minute_frame is not None:
        minute_line = chart_obj.create_line(
            name=minute_label,
            color=base_color,
            style='dashed',
            width=2,
            price_line=False,
            price_label=False,
        )
        minute_line.set(minute_frame)
        lines_added += 1

    return lines_added


if not results:
    print('Нет данных для визуализации')
else:
    if combine_on_one_canvas:
        chart = JupyterChart()
        any_lines = False
        for idx, (file_name, payload) in enumerate(results.items()):
            last_payload = payload
            added = add_equity_series(chart, payload, idx)
            if added == 0:
                print(f"{file_name}: нет данных для визуализации")
            any_lines = any_lines or added > 0
        if any_lines:
            chart.load()
    else:
        for idx, (file_name, payload) in enumerate(results.items()):
            chart = JupyterChart()
            last_payload = payload
            added = add_equity_series(chart, payload, idx)
            if added == 0:
                print(f"{file_name}: нет данных для визуализации")
                continue
            chart.load()

    if last_payload is None and results:
        last_key = next(reversed(results))
        last_payload = results[last_key]


In [ ]:

if last_payload:
    display(last_payload['events_equity'])
    display(last_payload['minute_equity'])
else:
    print('Нет рассчитанных данных для отображения')



## Альтернативная визуализация (Plotly)
Чтобы обойти ограничение по количеству точек у lightweight_charts, ниже добавлены вспомогательные ячейки для построения тех же эквити через Plotly. Эта библиотека спокойно отображает сотни тысяч точек, поэтому подходит для длительных периодов.


In [ ]:

# Построение графика для одного файла через Plotly
import plotly.graph_objects as go

def build_plotly_equity(payload: dict) -> go.Figure:
    fig = go.Figure()
    events = payload.get('events_equity')
    minute = payload.get('minute_equity')
    title = f"Equity для {payload['ticker']} ({payload['file']})"

    if events is not None and not events.empty:
        fig.add_trace(
            go.Scatter(
                x=events['datetime'],
                y=events['equity'],
                mode='lines',
                name=f"{payload['ticker']} equity (events)",
                line=dict(width=2)
            )
        )

    if minute is not None and not minute.empty:
        fig.add_trace(
            go.Scatter(
                x=minute['datetime'],
                y=minute['equity'],
                mode='lines',
                name=f"{payload['ticker']} equity (ohlc)",
                line=dict(width=1.5, dash='dash')
            )
        )

    fig.update_layout(
        title=title,
        xaxis_title='Время',
        yaxis_title='Equity',
        legend=dict(orientation='h', x=0.5, y=1.05, xanchor='center', yanchor='bottom'),
        hovermode='x unified',
    )
    return fig

if not results:
    print('Нет данных для визуализации — сначала выполните ячейки расчёта.')
else:
    first_key = next(iter(results))
    display(build_plotly_equity(results[first_key]))


In [ ]:

# Интерактивный выбор файла (если установлены ipywidgets)
try:
    import ipywidgets as widgets
except ImportError:
    widgets = None

if widgets is None:
    print('Module ipywidgets не найден. Установите его для интерактивного выбора (pip install ipywidgets).')
elif not results:
    print('Нет данных для визуализации — сначала выполните ячейки расчёта.')
else:
    options = list(results.keys())
    dropdown = widgets.Dropdown(options=options, description='Лог:')
    output = widgets.Output()

    def handle_change(change):
        if change.get('name') != 'value':
            return
        new_key = change.get('new')
        if new_key is None:
            return
        payload = results[new_key]
        fig = build_plotly_equity(payload)
        with output:
            output.clear_output(wait=True)
            display(fig)

    dropdown.observe(handle_change, names='value')
    display(dropdown)
    display(output)
    dropdown.value = options[0]
    handle_change({'name': 'value', 'new': dropdown.value})
